In [1]:
import os
from environs import Env

env = Env()
env.read_env(".env") # read .env file, if it exists

api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents import create_csv_agent
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts.chat import(
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

## Get list of accomodations

In [3]:
output_parser = CommaSeparatedListOutputParser()

In [4]:
format_instructions = output_parser.get_format_instructions()

In [5]:
agent = create_csv_agent(
    OpenAI(temperature=0, max_tokens=100, openai_api_key=api_key),
    "docs/student_roster.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

### Prompt for getting list of accomodations

In [6]:
message_for_accom = agent.invoke("List the accomodations for Abson Phoenix for Science.\n\n'Your response should be a list of comma separated values, eg: `foo, bar, baz`'")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the LAST column is Abson and the FIRST column is Phoenix and the SUBJECT column is Science.
Action: python_repl_ast
Action Input: df[(df['LAST'] == 'Abson') & (df['FIRST'] == 'Phoenix') & (df['SUBJECT'] == 'Science')]
Observation:      LAST    FIRST  SUBJECT                    ACCOMODATIONS
12  Abson  Phoenix  Science  Extra time, Oral administration
Thought: I need to select the ACCOMODATIONS column from this filtered dataframe.
Action: python_repl_ast
Action Input: df[(df['LAST'] == 'Abson') & (df['FIRST'] == 'Phoenix') & (df['SUBJECT'] == 'Science')]['ACCOMODATIONS']
Observation: 12    Extra time, Oral administration
Name: ACCOMODATIONS, dtype: object
Thought: I need to convert this series to a list.
Action: python_repl_ast
Action Input: list(df[(df['LAST'] == 'Abson') & (df['FIRST'] == 'Phoenix') & (df['SUBJECT'] == 'Science')]['ACCOMODATIONS'])
Observation: ['Extra time, Oral

In [7]:
accom_list = message_for_accom["output"].split(",")

## Write lesson plan

In [8]:
teks_code_schema = ResponseSchema(name="teks_code",
                             description="This is the TEKS code of the lesson plan.")

subject_schema = ResponseSchema(name="subject",
                                      description="This is the subject of the lesson plan.")

duration_schema = ResponseSchema(name="duration",
                                    description="This is the duration of the lesson plan.")

response_schemas = [teks_code_schema, 
                    subject_schema,
                    duration_schema]

In [9]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [10]:
format_instructions = """The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
    "teks_code": string  // This is the TEKS code of the lesson plan
    "subject": string  // This is the subject of the lesson plan
    "duration": string  // This is the duration of the lesson plan
    "warm_up_exercise": string // This is the warm-up exerciseportion of the lesson plan
    "attention_getter": string // This is the attention getter of the lesson plan
    "mini_lesson": string // This is the mini-lesson of the lesson plan
    "student_practice": string // This is the student practice of the lesson plan
    "exit_ticket": string // This is the exit ticket of the lesson plan
}
```
"""

In [11]:
template_string = "You are TeachGPT, an AI assistant that writes lessons plans according to the Texas Essential Knowledge and Skills (TEKS). Write a lesson plan on TEKS {teks_code} {subject}. The lesson plan must last for {duration} minutes. Include a warm-up exercise, an attention getter, a mini-lesson, student practice, and exit ticket.\n\n{format_instructions}"

In [12]:
prompt = ChatPromptTemplate.from_template(template=template_string)

### Prompt for lesson plan

In [13]:
messages = prompt.format_messages(teks_code="4.2C", 
                                  subject="Science",
                                  duration="60",
                                  format_instructions=format_instructions)

In [14]:
messages

[HumanMessage(content='You are TeachGPT, an AI assistant that writes lessons plans according to the Texas Essential Knowledge and Skills (TEKS). Write a lesson plan on TEKS 4.2C Science. The lesson plan must last for 60 minutes. Include a warm-up exercise, an attention getter, a mini-lesson, student practice, and exit ticket.\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\\`\\`\\`json" and "\\`\\`\\`":\n\n```json\n{\n    "teks_code": string  // This is the TEKS code of the lesson plan\n    "subject": string  // This is the subject of the lesson plan\n    "duration": string  // This is the duration of the lesson plan\n    "warm_up_exercise": string // This is the warm-up exerciseportion of the lesson plan\n    "attention_getter": string // This is the attention getter of the lesson plan\n    "mini_lesson": string // This is the mini-lesson of the lesson plan\n    "student_practice": string // This is the student pr

In [15]:
print(messages[0].content)

You are TeachGPT, an AI assistant that writes lessons plans according to the Texas Essential Knowledge and Skills (TEKS). Write a lesson plan on TEKS 4.2C Science. The lesson plan must last for 60 minutes. Include a warm-up exercise, an attention getter, a mini-lesson, student practice, and exit ticket.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
    "teks_code": string  // This is the TEKS code of the lesson plan
    "subject": string  // This is the subject of the lesson plan
    "duration": string  // This is the duration of the lesson plan
    "warm_up_exercise": string // This is the warm-up exerciseportion of the lesson plan
    "attention_getter": string // This is the attention getter of the lesson plan
    "mini_lesson": string // This is the mini-lesson of the lesson plan
    "student_practice": string // This is the student practice of the lesson plan
    "exit_ticke

In [16]:
chat_llm = ChatOpenAI(temperature=0.0)

In [17]:
response = chat_llm.invoke(messages)

In [18]:
response_dict = output_parser.parse(response.content)

In [19]:
response_dict['accomodations'] = accom_list

In [20]:
response_dict

{'teks_code': '4.2C',
 'subject': 'Science',
 'duration': '60 minutes',
 'warm_up_exercise': 'Ask students to name different types of materials they encounter in their daily lives (e.g., wood, plastic, metal, glass).',
 'attention_getter': 'Show students a variety of objects made from different materials (e.g., a wooden pencil, a plastic bottle, a metal spoon, a glass jar). Ask them to observe and discuss the materials used to make each object.',
 'mini_lesson': 'Explain to students that materials have different properties that make them suitable for specific purposes. Discuss common properties such as hardness, flexibility, transparency, and conductivity. Show examples of materials that exhibit these properties (e.g., a hard rock, a flexible rubber band, a transparent glass window, a conductive metal wire).',
 'student_practice': 'Divide students into small groups and provide each group with a set of materials (e.g., paper, fabric, plastic, metal). Instruct them to explore the propert

## Create worksheet differentiated according to accomodations of the student

In [21]:
worksheet_template_string = "Create a worksheet to go along with {student_practice} and differentiate according to the {accom_list} needed."

In [22]:
prompt = ChatPromptTemplate.from_template(template=worksheet_template_string)

messages = prompt.format_messages(student_practice=response_dict["student_practice"],
                                  accom_list=response_dict["accomodations"])

In [23]:
print(messages[0].content)

Create a worksheet to go along with Divide students into small groups and provide each group with a set of materials (e.g., paper, fabric, plastic, metal). Instruct them to explore the properties of each material and record their observations. Encourage them to test the materials by bending, stretching, tearing, and observing how they interact with water or other objects. and differentiate according to the ['Extra time', ' Oral administration'] needed.


In [24]:
worksheet_response = chat_llm.invoke(messages)
worksheet_response

AIMessage(content='Worksheet: Exploring Material Properties\n\nGroup Name: _______________________\n\nMaterials Provided: _______________________\n\nInstructions:\n1. In your group, explore the properties of each material provided.\n2. Use the materials to perform various tests such as bending, stretching, tearing, and observing their interaction with water or other objects.\n3. Record your observations in the table below.\n4. Discuss your findings as a group and answer the questions at the end.\n\nMaterial Properties Observation Table:\n\nMaterial | Bending | Stretching | Tearing | Interaction with Water/Objects\n---------|---------|------------|---------|-------------------------------\n1.       |         |            |         |                               \n2.       |         |            |         |                               \n3.       |         |            |         |                               \n4.       |         |            |         |                               

In [25]:
x = str(worksheet_response).replace("\\n", "\n")

In [26]:
print(x)

content='Worksheet: Exploring Material Properties

Group Name: _______________________

Materials Provided: _______________________

Instructions:
1. In your group, explore the properties of each material provided.
2. Use the materials to perform various tests such as bending, stretching, tearing, and observing their interaction with water or other objects.
3. Record your observations in the table below.
4. Discuss your findings as a group and answer the questions at the end.

Material Properties Observation Table:

Material | Bending | Stretching | Tearing | Interaction with Water/Objects
---------|---------|------------|---------|-------------------------------
1.       |         |            |         |                               
2.       |         |            |         |                               
3.       |         |            |         |                               
4.       |         |            |         |                               
5.       |         |        